# Imports

In [1]:
import pandas as pd
import numpy as np
import copy
import scanpy as sc

# Read in Data

In [33]:
# read in vdj info, make easily searchable
vdj = pd.read_csv('../../projects/2024_11_20_10X11_14234_0/out/outs/per_sample_outs/2024_11_20_10X11_14234_0_multi/vdj_b/filtered_contig_annotations.csv')

for i in range(len(vdj['v_gene'])):
    ig = vdj['v_gene'][i][2:]
    chain = ig[0]
    family = ig[2]
    # gene = ig.split('-')[1]
    
    vdj.loc[i, 'chain'] = chain
    vdj.loc[i, 'family'] = family
    # vdj.loc[i, 'gene'] = gene

# read in mtx data from gex analysis
mat = sc.read_10x_mtx('../../projects/2024_11_20_10X11_14234_0/out/outs/per_sample_outs/2024_11_20_10X11_14234_0_multi/count/sample_filtered_feature_bc_matrix')
gex = mat.to_df()
gex = gex.reset_index(names='cell_id')
gex.shape


(16896, 38607)

In [43]:
len(vdj)

23266

In [42]:
len(vdj.barcode.unique())

12168

In [57]:
sum([gex.loc[i,'cell_id'] in set(vdj.barcode) for i in range(len(gex))])

12168

# Functions

In [36]:
# create cell selection function
def select_cells(chain, family, vdj=vdj, gex=gex):
    '''
    Selects the intersction between airr and gex when using chain and family from vdj
    '''
    # vdj_filtered = vdj[(vdj[vdj['chain']==chain]) & (vdj['family']==family)]
    vdj_chain = vdj[vdj['chain']==chain]
    vdj_fam = vdj_chain[vdj_chain['family'] == family]
    intersection = vdj_fam.merge(gex, how='inner', left_on='barcode', right_on='cell_id')
    return intersection

def select_gene(df, gene):
    '''
    Provide a gene, receive tuple with negatives or positivies. 
    tuple[0] == false, tuple[1] == true
    '''
    neg = df[df[gene]==0]
    pos = df[df[gene]>0]

    return (neg, pos)

def compare(gene, type1='IGHV3', type2='IGHV4'):

    print(f'Selecting {type1} cells...')
    cells1 = select_cells(type1[2], type1[4])
    print(f'Selecting {type2} cells...')
    cells2 = select_cells(type2[2], type2[4])

    print(f'Selecting for {gene} in {type1} cells...')
    selected1 = select_gene(cells1, gene)
    print(f'Selecting for {gene} in {type2} cells...')
    selected2 = select_gene(cells2, gene)

    print('='*26)
    print(f'{gene:<8} {"+":<8} {"-":<8}')
    print('-'*26)
    print(f'{type1:<8} {len(selected1[1]):<8} {len(selected1[0]):<8}')
    print(f'{type2:<8} {len(selected2[1]):<8} {len(selected2[0]):<8}\n')
    print('='*26+'\n')

def compare_selected(gene, cells1, cells2):
    selected1 = print(f'Selecting for {gene}...')
    selected1 = select_gene(cells1, gene)
    selected2 = select_gene(cells2, gene)

    name1 = 'IG'+cells1['chain'][0]+'V'+cells1['family'][0]
    name2 = 'IG'+cells2['chain'][0]+'V'+cells2['family'][0]

    print('='*26)
    print(f'{gene:<8} {"+":<8} {"-":<8}')
    print('-'*26)
    print(f'{name1:<8} {len(selected1[1]):<8} {len(selected1[0]):<8}')
    print(f'{name2:<8} {len(selected2[1]):<8} {len(selected2[0]):<8}')
    print('='*26+'\n')

def create_clusters(group_name, genes, cells, filename='local'):
    '''
    Creates clusters for loupe browser with list of genes, but on ighv3 and ighv4
    '''
    if filename == 'local':
        filename = '&'.join(group_name.split('/'))+'.csv'
    genes = ['XBP1', 'LILRB1']

    out = pd.DataFrame()
    out['Barcode'] = pd.concat([ighv3, ighv4], ignore_index=True)['cell_id']
    out[group_name] = ['negative']*len(out['Barcode'])
    out = out.drop_duplicates(ignore_index=True)
    # out.head()
    for gene in genes:
        gene_and_cells = [select_gene(cell, gene) for cell in cells]
        # h3gene = select_gene(ighv3, gene)
        # h4gene = select_gene(ighv4, gene)
        bothgenepos = pd.concat([e[1] for e in gene_and_cells], ignore_index=True)
        
        for i in range(len(out)):
            if out.loc[i,'Barcode'] in bothgenepos['cell_id'].values:
                if out.loc[i,group_name] != 'negative':
                    out.loc[i,group_name] = out.loc[i,group_name] + '/' + gene
                else:
                    out.loc[i,group_name] = gene

    out.to_csv(filename, index=False)

# Testing

In [4]:
ighv3 = select_cells('H', '3')
ighv4 = select_cells('H', '4')

In [6]:
# ighv3 = select_cells('H', '3')
# ighv4 = select_cells('H', '4')

# irf4 = select_gene(ighv4, 'IRF4')
# print(len(irf4[1]), len(irf4[0]), len(irf4[1]) + len(irf4[0]))

compare('IRF4')


Selecting IGHV3 cells...
Selecting IGHV4 cells...
Selecting for IRF4 in IGHV3 cells...
Selecting for IRF4 in IGHV4 cells...

IRF4     +        -       
--------------------------
IGHV3    404      4345    
IGHV4    586      3840    


### Plasmablast
Positive marker indicates plasmablast

In [19]:
# Better marker for plasmablast? Higher count than IRF4
compare_selected('IRF4', ighv3, ighv4)
compare_selected('XBP1', ighv3, ighv4)

Selecting for IRF4...
--------------------------
IRF4     +        -       
IGHV3    404      4345    
IGHV4    586      3840    
--------------------------

Selecting for XBP1...
--------------------------
XBP1     +        -       
IGHV3    732      4017    
IGHV4    705      3721    
--------------------------



### Memory B Cells
Positive marker indicates memory b cell

In [ ]:
compare_selected('TLR9', ighv3, ighv4) # Memory B
# compare_selected('TNFRSF13B', ighv3, ighv4) # Mature B, Memory B, Plasma Cell
compare_selected('NT5E', ighv3, ighv4) # Memory B
compare_selected('LILRB1', ighv3, ighv4) # Atypical Memory B
compare_selected('FCRL4', ighv3, ighv4) # Atypical Memory B


Selecting for TLR9...
TLR9     +        -       
--------------------------
IGHV3    79       4670    
IGHV4    260      4166    

Selecting for NT5E...
NT5E     +        -       
--------------------------
IGHV3    433      4316    
IGHV4    213      4213    

Selecting for LILRB1...
LILRB1   +        -       
--------------------------
IGHV3    793      3956    
IGHV4    1016     3410    

Selecting for FCRL4...
FCRL4    +        -       
--------------------------
IGHV3    4        4745    
IGHV4    211      4215    



### Output file
For visualization in Loupe Browser

In [ ]:
# creates a csv that can be imported into loupe browser as a custom cluster
# Combines IGHV3 and IGHV4 cells and shows LILRB1+ (atypical mbc), XBP1+ (pb), both +, or both -
# in ighv3/ighv4 LILRB1, XBP1
genes = ['XBP1', 'LILRB1']

out = pd.DataFrame()
out['Barcode'] = pd.concat([ighv3, ighv4], ignore_index=True)['cell_id']
out['PB/MBC in IGHV3/4'] = ['negative']*len(out['Barcode'])
out = out.drop_duplicates(ignore_index=True)
# out.head()
for gene in genes:
    h3gene = select_gene(ighv3, gene)
    h4gene = select_gene(ighv4, gene)
    bothgenepos = pd.concat([h3gene[1], h4gene[1]], ignore_index=True)
    
    for i in range(len(out)):
        if out.loc[i,'Barcode'] in bothgenepos['cell_id'].values:
            if out.loc[i,'PB/MBC in IGHV3/4'] != 'negative':
                out.loc[i,'PB/MBC in IGHV3/4'] = out.loc[i,'PB/MBC in IGHV3/4'] + '/' + gene
            else:
                out.loc[i,'PB/MBC in IGHV3/4'] = gene

out.to_csv('PB&MBC in IGHV3&4.csv', index=False)


In [37]:
create_clusters('PB/MBC in IGHV3/4', ['XBP1', 'LILRB1'], [ighv3, ighv4])

# Scratch

In [30]:
# find if barcode from ighv3 in ighv4  
for i in range(len(ighv3)):
    if ighv3.loc[i, 'cell_id'] in ighv4['cell_id'].values:
        print(i)

130
201
829
848
1062
1529
1913
1921
1926
1999
2001
2231
2238
3245
3787
3878
4510


In [ ]:
cd19 = select_gene(gex, 'CD19') # cd19 +/-


In [ ]:
cd27_cd19 = select_gene(cd19[1], 'CD27') # cd27 +/- given cd19+

In [ ]:

cd38_cd27_cd19 = select_gene(cd27_cd19[1], 'CD38') # cd38 +/- given cd27+ given cd19+

In [ ]:
cd38 = select_gene(gex, 'CD38')


In [ ]:
len(cd38[1])

In [5]:
# create df of irf4+ and vcall


# create df of irf4- and vcall


# count based on vcall

In [8]:
# # setup
# cell = pd.DataFrame({'cell_id':[],
#                       'v_call':[]})

# fam_dicts = dict([(e, None) for e in list(set([e[4] for e in vdj['v_call'].unique()]))])

# cells = {'H':copy.deepcopy(fam_dicts),
#          'K':copy.deepcopy(fam_dicts),
#          'L':copy.deepcopy(fam_dicts)}

# # sort cells by chain and family it to dict of dicts sub structure
# for i in range(len(vdj['v_call'])):
#     ig = vdj['v_call'][i][2:5]
#     chain = ig[0]
#     family = ig[-1]

#     if type(cells[chain][family]) != type(pd.DataFrame()):
#         cells[chain][family] = cell.copy(deep=True)
    
#     cells[chain][family].loc[i] = {'cell_id':vdj['cell_id'][i], 
#                                    'v_call':vdj['v_call'][i]}

# # cells['H']['4']
